In [1]:
import pandas as pd
import requests

In [2]:
data = pd.read_csv('../../../data/kc_house_data_update4.csv')

In [3]:
latlongs = data['lat'].map(str) + ',' + data['long'].map(str)
latlongs

0                   47.5112,-122.257
1        47.721000000000004,-122.319
2        47.7379,-122.23299999999999
3        47.5208,-122.39299999999999
4                   47.6168,-122.045
                    ...             
21579               47.6993,-122.346
21580    47.5107,-122.36200000000001
21581    47.5944,-122.29899999999999
21582               47.5345,-122.069
21583    47.5941,-122.29899999999999
Length: 21584, dtype: object

In [4]:
latlongs = list(latlongs)
subsets = []
current_start = 0
for x in range(len(latlongs)//500):
    subset = latlongs[current_start:current_start+500]
    subsets.append(subset)
    current_start += 500
subset = latlongs[-84:]
subsets.append(subset)

In [5]:
sizes = [len(subset) for subset in subsets]
sum(sizes)

21584

In [6]:
points = ['|'.join(subset) for subset in subsets]

In [7]:
all_calls = []
import time
for subset in points:
    url = f'https://maps.googleapis.com/maps/api/elevation/json?locations={subset}&key=YOUR-KEY-HERE'

    payload = {}
    headers = {}

    response = requests.request('get', url, headers = headers, data = payload)

    all_calls.append(response.json())

#### NOTE: The above code cell will not work without an API key.
If you as a reader would like to run it, insert an API key into the url where it says `YOUR-KEY-HERE`. Otherwise, the `all_calls` and `conglomerate` objects seen later in the code are located in `notebooks/exploratory/Andre/Scraped` as objects pickled with the `pickle` library.

In [8]:
conglomerate = []
for call in all_calls:
    conglomerate += call['results']

In [9]:
len(conglomerate)

21584

In [10]:
import pickle
with open('Scraped/all_calls.pickle', 'wb') as f:
    pickle.dump(all_calls, f, pickle.HIGHEST_PROTOCOL)
    
with open('Scraped/conglomerate.pickle', 'wb') as f:
    pickle.dump(conglomerate, f, pickle.HIGHEST_PROTOCOL)

In [11]:
elevations = []
lats = []
longs = []
for point in conglomerate:
    elevations.append(point['elevation'])
    lats.append(point['location']['lat'])
    longs.append(point['location']['lng'])

In [12]:
import numpy as np
lats = np.array(lats)
longs = np.array(longs)

In [13]:
abs((lats - data['lat'])).max()

7.105427357601002e-15

In [14]:
abs((longs - data['long'])).max()

1.4210854715202004e-14

In [15]:
data['elevation'] = elevations

In [16]:
data[['lat','long','elevation']]

,lat,long,elevation
0,47.5112,-122.257,86.437805
1,47.7210,-122.319,106.938599
2,47.7379,-122.233,147.442520
3,47.5208,-122.393,15.198629
4,47.6168,-122.045,161.984726
...,...,...,...
21579,47.6993,-122.346,94.916382
21580,47.5107,-122.362,94.977394
21581,47.5944,-122.299,66.968323
21582,47.5345,-122.069,151.926895


In [17]:
data.to_csv(path_or_buf = '../../../data/kc_house_data_update5.csv', index = False)